In [ ]:
# from pathlib2 import *
from pathlib import *
import json

import os
import shutil as sh
import tensorflow as tf
import sys

sys.path.insert(0, "/workspace/models/slim")
# sys.path.insert(0, "/Users/wcz/Beanflows/All_Beans/Machine_Learning/innovavant-Training-Car-Recognition-Model/choose-network-architecture/models/slim")


from eval_image_classifier import main

In [ ]:
!pwd

In [ ]:
# for all/chosen folders
# for all/chosen checkpoints in a folder
# use ckpt file
# from eval_image_classifier import ???
# run it for one checkpoint, one training or list of trainings

# get_list_of_training_folders

In [ ]:
def get_list_of_checkpoints_for_training(train_log_path):
    items = list(train_log_path.iterdir())

    # filter ckpt out
    ckpts = filter(lambda item: 'ckpt' in item.name, items)

    # filter index out
    ckpts = filter(lambda item: 'index' in item.suffix, items)

    # get number ... .ckpt-n. ..., cut 'ckpt-' out
    def ckpt_num(name):
        name = name.split('.',2)[1]
        ckpt_num = int(name.split('-',2)[1])
        return ckpt_num

    # create list of checkpoint numbers, sort it
    ckpt_nums = sorted(map(lambda item: ckpt_num(item.name), ckpts))
    return ckpt_nums

    # map it into "model.ckpt-n"
#     ckpts = map(lambda num: 'model.ckpt-{}'.format(num), ckpt_nums)
#     return ckpts

In [ ]:
last_eval_checkpoint_key = 'last_eval_checkpoint'

def load_last_eval_checkpoint(EVAL_DIR):
    fn = str(EVAL_DIR / last_eval_checkpoint_key)

    try:
        with open(fn, 'r') as f:
            try:
                data = json.load(f)
            # if the file is empty the ValueError will be thrown
            except ValueError:
               data = {last_eval_checkpoint_key : -1}
    except IOError: # python 2
#     except FileNotFoundError: # python 3
               data = {last_eval_checkpoint_key : -1}

        
    return data[last_eval_checkpoint_key]

def save_last_eval_checkpoint(EVAL_DIR, last_eval_checkpoint):
    fn = str(EVAL_DIR / last_eval_checkpoint_key)
    json.dump({last_eval_checkpoint_key : last_eval_checkpoint}, open(fn,'w'))

In [ ]:
# RUN = '2_car-reco3_inception_v3-mac' # model_name = 'inception_v3'
# RUN = '3_car-reco3_inception_v3' # model_name = 'inception_v3'
# RUN = '4_car-reco3_inception_resnet_v2' # model_name = 'inception_resnet_v2'
# RUN = '5_car-reco3_inception_v4' # model_name = 'inception_v4'
# RUN = '6_car-reco3_resnet_v2_152' # model_name = 'resnet_v2_152'
# RUN = '7_car-reco3_resnet_v2' # model_name = 'resnet_v1_152'
# RUN = '8_car-reco3_inception_v3' # model_name = 'inception_v3'
# -----------------------------------------------
# RUN = '9_car-reco3-70_inception_v3' # model_name = 'inception_v3'
# RUN = '10_car-reco3-70_inception_v4' # model_name = 'inception_v4'
# RUN = '11_car-reco3-70_inception_resnet_v2' # model_name = 'inception_resnet_v2'
# RUN = '13_car-reco3-70_inception_resnet_v2' # model_name = 'inception_resnet_v2'

In [ ]:
FLAGS = tf.app.flags.FLAGS

# RUN = '12_car-reco3-70_resnet_v2_152'
# model_name = 'resnet_v2_152'

# RUN = '15_car-reco3-70_resnet_v2_152'
# model_name = 'resnet_v2_152'

# RUN = '16_comp_cars_resnet_v2_152'
# model_name = 'resnet_v2_152'

RUN = '17_car-reco3-70_resnet_v2_152'
model_name = 'resnet_v2_152'



# max_num_batches    = 10 # 10 # 43 # 435
max_num_batches    = 179 # 4 # 50 # 179
BASE_DIR           = Path('/workspace')
# BASE_DIR           = Path('/Users/wcz/Beanflows/All_Beans/Machine_Learning/innovavant-Training-Car-Recognition-Model/choose-network-architecture')
TRAIN_DIR          = BASE_DIR / 'big_storage/all_train_logs' / RUN
EVAL_DIR           = TRAIN_DIR / 'eval-{}'.format(max_num_batches)
DATASET_DIR        = BASE_DIR / 'data' / 'car-reco3-70'
# DATASET_DIR        = BASE_DIR / 'data' / 'comp_cars'
dataset_name       = 'car_reco3_70'
# dataset_name       = 'comp_cars'

# dataset_split_name = 'train'
dataset_split_name = 'validation'

tf.gfile.MakeDirs(str(EVAL_DIR))

print(EVAL_DIR);

In [ ]:
all_ckpt_nums = get_list_of_checkpoints_for_training(TRAIN_DIR)
ckpt_count = len(all_ckpt_nums)
print ('ckpt_count: ', ckpt_count)

last_eval_checkpoint = load_last_eval_checkpoint(EVAL_DIR)
# get checkpoint numbers bigger than last_checkpoint_number
ckpt_nums = list(filter(lambda x: x > last_eval_checkpoint, all_ckpt_nums))
ckpt_nums = [all_ckpt_nums[-1]]

print('all_ckpt_nums: ', all_ckpt_nums)
print('last_eval_checkpoint: ', last_eval_checkpoint)
print('ckpt_nums: ', ckpt_nums)

In [ ]:
checkpoints = map(lambda num: 'model.ckpt-{}'.format(num), ckpt_nums)

FLAGS.eval_dir           = str(EVAL_DIR)
FLAGS.dataset_name       = dataset_name
FLAGS.dataset_split_name = dataset_split_name
FLAGS.model_name         = model_name
FLAGS.max_num_batches    = max_num_batches

for ckpt_num in ckpt_nums:
    checkpoint_name = 'model.ckpt-{}'.format(ckpt_num)
    FLAGS.dataset_dir        = str(DATASET_DIR)
    FLAGS.checkpoint_path    = str(TRAIN_DIR / checkpoint_name)
    print(FLAGS.checkpoint_path)
    #FLAGS.labels_offset      = 1
    main(FLAGS)
    save_last_eval_checkpoint(EVAL_DIR, ckpt_num)